In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
from sklearn.metrics import confusion_matrix
from src.utils import DatasetUtils

c:\users\aakaas~1\docume~1\nyu\projects\speech~1\env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
keras = tf.keras
py.init_notebook_mode(connected=True)
utils = DatasetUtils()
encoder = utils.get_label_encoder()

In [3]:
test_data, test_labels = utils.get_dataset_and_encoded_labels('test_data.npy', 'test_labels.npy')
unrolled_labels = np.load('./data/test_labels_unrolled.npy')

In [4]:
n_arch1 = keras.models.load_model('./models/noisy/convnet.h5')
# n_arch2 = keras.models.load_model('./models/noise_based_models/ResNet.h5')
# n_arch3 = keras.models.load_model('./models/noise_based_models/RRNN.h5')

In [5]:
n_arch1_pred = n_arch1.predict(test_data)
# n_arch2_pred = n_arch2.predict(test_data)
# n_arch3_pred = n_arch3.predict(test_data)

In [6]:
n_arch1_labels = encoder.classes_[np.argmax(n_arch1_pred, axis=-1)]
# n_arch2_labels = encoder.classes_[np.argmax(n_arch2_pred, axis=-1)]
# n_arch3_labels = encoder.classes_[np.argmax(n_arch3_pred, axis=-1)]
ground_truth_labels = encoder.classes_[np.argmax(test_labels, axis=-1)]

In [7]:
df = pd.DataFrame({'unrolled_gt': unrolled_labels, 'masked_gt': ground_truth_labels, 'pred': n_arch1_labels})

In [8]:
df_unknown = df[df['masked_gt'] == 'unknown'][['unrolled_gt', 'pred']]

In [9]:
df_dropped_unknowns = df_unknown[df_unknown['pred'] != 'unknown']

In [10]:
df_dropped_unknowns['value'] = np.ones(len(df_dropped_unknowns))
table = pd.pivot_table(df_dropped_unknowns, values='value', index=['unrolled_gt'], columns=['pred'], aggfunc=np.sum)
table = table.fillna(value=0)
py.iplot([go.Heatmap(z=table.values, x=table.columns, y=table.index)], filename='n_arch1_confusion_matrix')

c:\users\aakaas~1\docume~1\nyu\projects\speech~1\env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

